In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
df = pd.read_excel('/Users/surjitdas/Box Sync/Barclays Data Tower/Governance/Operations/Resource Master - IIT.xlsx','Base data')

In [2]:
df.columns

Index(['Contract', 'WP_#', 'Emp_Code', 'First_Name', 'Last_Name',
       'Emp_Notes_ID', 'Band', 'Band_Wt', 'Status', 'Demand_Confidence',
       'Supply_Status', 'Demand_Received', 'Requested_Start_date', 'Billable',
       'NCE_Start_Date', 'NCE_End_Date', 'Transition_Start_Date',
       'Transition_End_Date', 'Billing_Start_Date', 'Billing_End_Date',
       'Billing_Month', 'Billing_Year', 'Billing_End_Month', 'Billing_End_Yr',
       'Unbilled_Days', 'Scoped_vs_Non_Scoped', 'SR_#', 'SR_Start_Date',
       'SR_End_Date', 'BRID', 'BRID_Date', 'Line_Manager', 'IBM POC',
       'Location_Category', 'Skills', 'JRSS', 'Premium', 'PSL_Role',
       'Years_of_Experience', 'Role_Category', 'BCP Critical', 'Comments',
       'Inactive_Reason', 'City'],
      dtype='object')

In [3]:
df.shape

(539, 44)

# Reporing queries:

In [17]:
#1. Syam T&M Headcount
print(df.groupby(["Status","Scoped_vs_Non_Scoped","Line_Manager"])['Emp_Notes_ID'].count())

Status    Scoped_vs_Non_Scoped  Line_Manager                     
Active    Non-scoped            Ajay Shelke                           1
                                Amit Sonar                            1
                                Ankit Tharwani                        6
                                Anusha Anandraj                       2
                                Binod Maveli                          1
                                Chandrasekar Chandramohan             8
                                Jyoti Bharadwaj                      15
                                Karthik Ranganathan                   1
                                Koti Reddy                            1
                                Krishna Saboo                         1
                                Madhumita Deo                         1
                                Mahesh Dekhane                        9
                                Manish Doshi                          

In [ ]:
df("Line_Manager")

In [ ]:
#3. How many (and which) Active Emp Notes ID do not have /IBM (proper notes id)
df[(~df['Emp_Notes_ID'].str.endswith("IBM")) & (df['Status'] == 'Active')]

In [ ]:
#4. How many (and which) Active Emp Codes are missing?
df[(df['Emp_Code'].isnull()) & (df['Status'] == 'Active')]

In [ ]:
#5. How many (and which) Bands are missing?
df[(df['Band'].isnull())]

In [ ]:
#6. How many (and which) Line Managers are missing?
df[(df['Line_Manager'].isnull())]

In [ ]:
#7. How many (and which) JRSS are missing?
df[(df['JRSS'].isnull())]

In [ ]:
#8. How many (and which) Active Years of Experience are missing?
df[(df['Years_of_Experience'].isnull()) & (df['Status'] == 'Active')]

In [ ]:
#9. How many (and which) Active PSL Role are missing?
df[(df['PSL_Role'].isnull()) & (df['Status'] == 'Active')]

In [ ]:
#10. How many (and which) Active Role Category are missing?
df[(df['Role_Category'].isnull()) & (df['Status'] == 'Active')]

In [ ]:
#11. How many (and which) Active City are missing?
df[(df['City'].isnull()) & (df['Status'] == 'Active')]

In [ ]:
#12. Which Inactive records have Inactive_Reason missing?
df[(df['Status'] == 'Inactive') & df.Inactive_Reason.isnull()]

# Data Updation issues:

In [ ]:
from datetime import datetime,timedelta
now = datetime.now()

#1. Which Active resources have Billing End Date in the next 30 days?
df[(~df['Billing_End_Date'].isnull()) & (df['Billing_End_Date'] < now + timedelta(days=30))& (df['Status'] == 'Active')]

In [ ]:
#2. Which Active resources’ Billing End Date is in the past?
df[(~df['Billing_End_Date'].isnull()) & (df['Billing_End_Date'] < now)& (df['Status'] == 'Active')]

In [ ]:
#3. Which Forecast resources’ NCE Start Date is in the past?
df[(~df['NCE_Start_Date'].isnull()) & (df['NCE_Start_Date'] < now)& (df['Status'] == 'Forecast')]

In [ ]:
#4. Which Forecast resources’ Transition Start Date is in the past?
df[(~df['Transition_Start_Date'].isnull()) & (df['Transition_Start_Date'] < now)& (df['Status'] == 'Forecast')]

In [ ]:
#5. Which Forecast resources’ Billing Start Date is in the past?
df[(~df['Billing_Start_Date'].isnull()) & (df['Billing_Start_Date'] < now)& (df['Status'] == 'Forecast')]

In [ ]:
#6. Which Inactive resources’ Billing End Date is in the future or is missing?
df[((df['Billing_End_Date'].isnull()) | (df['Billing_End_Date'] > now)) & (df['Status'] == 'Inactive')]

## Key insights:

In [ ]:
#1. List the key Ab Initio Leads (7B & above)
df[(df.Band.isin(['7B',8,9])) & (df['Status'] == 'Active')  & (df['Skills'] == 'ETL Ab Initio')]

In [ ]:
#2. List all other leads (7B & above)
df[(df.Band.isin(['7B',8,9])) & (df['Status'] == 'Active')  & (df['Skills'] != 'ETL Ab Initio')]

In [ ]:
#3. List potential Ab Initio Leads (7A)
df[(df.Band=='7A') & (df['Status'] == 'Active')  & (df['Skills'] == 'ETL Ab Initio')]

In [ ]:
#4. List potential other Leads (7A)
df[(df.Band=='7A') & (df['Status'] == 'Active')  & (df['Skills'] != 'ETL Ab Initio')]

In [ ]:
#5. What is the Premium % (Active and with Forecast)
#Create a pandas.core.groupby.DataFrameGroupBy object based on the Grouping required
#The following provides a grouping on (["Status","Premium"]) columns
#We can further filter the return columns in the same statement by adding [["Emp_Notes_ID","Band"]]
#gb = df.groupby(["Status","Premium"])[["Emp_Notes_ID","Band_Wt"]]

#But we will break this up into multiple steps for easy calculations
gb = df.groupby(["Status","Premium"])
Active_Premium = gb.get_group(('Active','Premium'))["Emp_Notes_ID"].count()
Active_PremiumPlus = 0 #gb.get_group(('Active','Premium Plus'))["Emp_Notes_ID"].count()
Active_Standard = gb.get_group(('Active','Standard'))["Emp_Notes_ID"].count()
Forecast_Premium = gb.get_group(('Forecast','Premium'))["Emp_Notes_ID"].count()
Forecast_Standard = gb.get_group(('Forecast','Standard'))["Emp_Notes_ID"].count()

Active_Premium_Pct = (Active_Premium + Active_PremiumPlus)*100 / (Active_Premium + Active_PremiumPlus + Active_Standard)
Active_plus_Forecast_Premium_Pct = (Active_Premium + Active_PremiumPlus + Forecast_Premium)*100 / (Active_Premium + Active_PremiumPlus + Active_Standard + Forecast_Premium + Forecast_Standard)

Active_Premium_Pct, Active_plus_Forecast_Premium_Pct

In [ ]:
#6. What is the Band mix (Active and with Forecast)
gb = df.groupby(["Status"])
Active_Band_Wt = gb.get_group(('Active'))['Band_Wt'].mean()
Forecast_Band_Wt = gb.get_group(('Forecast')).Band_Wt.mean()
Active_plus_Forecast_Band_Wt = (gb.get_group(('Active')).Band_Wt.sum() + gb.get_group(('Forecast')).Band_Wt.sum()) / (gb.get_group(('Active')).Band_Wt.count() + gb.get_group(('Forecast')).Band_Wt.count())
Active_Band_Wt, Active_plus_Forecast_Band_Wt

In [ ]:
#7. What is the skills distribution?
df.groupby(["Status","Scoped_vs_Non_Scoped","Skills"])['Emp_Notes_ID'].count()

In [ ]:
#Revenue Calculations
thisYr = 2017
first_date = pd.Series({1:date(thisYr,1,1), 2:date(thisYr,2,1), 3:date(thisYr,3,1), 4:date(thisYr,4,1), 5:date(thisYr,5,1), 6:date(thisYr,6,1), 7:date(thisYr,7,1), 8:date(thisYr,8,1), 9:date(thisYr,9,1), 10:date(thisYr,10,1), 11:date(thisYr,11,1), 12:date(thisYr,12,1)})
last_date = pd.Series({1:date(thisYr,1,31), 2:date(thisYr,2,28), 3:date(thisYr,3,31), 4:date(thisYr,4,30), 5:date(thisYr,5,31), 6:date(thisYr,6,30), 7:date(thisYr,7,31), 8:date(thisYr,8,31), 9:date(thisYr,9,30), 10:date(thisYr,10,31), 11:date(thisYr,11,30), 12:date(thisYr,12,31)})
local_billing_rate = {1:990, 2:990, 3:990, 4:990, 5:990, 6:990, 7:990, 8:990, 9:990, 10:990, 11:990, 12:990}
landed_billing_rate = {1:455, 2:455, 3:455, 4:455, 5:455, 6:455, 7:455, 8:455, 9:455, 10:455, 11:455, 12:455}
offshore_billing_rate = {1:121, 2:121, 3:121, 4:121, 5:121, 6:121, 7:121, 8:121, 9:121, 10:121, 11:121, 12:121}
billingRate = {"Local":local_billing_rate, "Landed":landed_billing_rate, "Offshore":offshore_billing_rate}
assumed_ute = 0.9

def getDaysForMonth(startDate, endDate, forMonth):

    if startDate == date.min:
        return 0
    elif startDate > last_date[forMonth]:
        return 0
    elif startDate < first_date[forMonth]:
        startDate = first_date[forMonth]
    
    if endDate == date.max:
        endDate = last_date[forMonth]
    elif endDate < first_date[forMonth]:
        return 0
    elif endDate > last_date[forMonth]:
        endDate = last_date[forMonth]    
    
    delta = np.busday_count(startDate, endDate)
    return delta * assumed_ute

def getRevenue (startDate, endDate, locationCategory):
    revenueByMonth = {}
    for month in range(1,13):
        revenueByMonth[month] = getDaysForMonth(startDate, endDate, month) * billingRate[locationCategory][month]
    return revenueByMonth

df2 = pd.DataFrame()
for index, row in df.iterrows():
    nceStart = row['NCE_Start_Date']
    nceEnd = row['NCE_End_Date']
    transitionStart = row['Transition_Start_Date']
    transitionEnd = row['Transition_End_Date']
    billingStart = row['Billing_Start_Date']
    billingEnd = row['Billing_End_Date']
    locationCategory = row['Location_Category']
    
    if pd.isnull(billingEnd): #Very important technique of finding null -- pd.isnull(xxx)
        billingEnd = datetime(thisYr,12,31,0,0,0)
        
    revSer = getRevenue(billingStart.date(), billingEnd.date(),locationCategory)
    df2 = df2.append([revSer],index) #Just a df2.append([..]) does not work. Has to be df2 = df2...

df3 = df.join(df2)
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('/Users/surjitdas/Box Sync/Barclays Data Tower/Governance/Operations/calculations.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df3.to_excel(writer, sheet_name='Revenue')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
print("Done")

In [ ]:
df[(df['Status'] == 'Forecast')  & (df.Confidence=='Identified') ]